In [28]:
import zipfile
import pandas as pd
import numpy as np
import xarray as xr

In [105]:
zip_file_path = 'vcl_ts_-74.14687_-73.61269_40.08408_40.49947.zip'
csv_filename = 'vcl_ts_-74.14687_-73.61269_40.08408_40.49947.csv'
save_to = '.'

In [106]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extract(csv_filename, '.')

In [116]:
df = pd.read_csv('vcl_ts_-74.14687_-73.61269_40.08408_40.49947.csv', header=13, index_col=0)
if len(df['dx'].unique())==1:
    df = df.drop(['dx'], axis=1)

In [117]:
var1 = 'wsp'
var2 = 'wdir'
df_wsp, df_wdir = (pd.DataFrame(index=df.index) for _ in range(2))
altitude = []

for col in df:
    if var1 in col:
        altitude.append(col.replace(var1+'_', ""))
        df_wsp[col] = df[col]
    elif var2 in col:
        df_wdir[col] = df[col]
        
height = np.array(altitude, dtype = float)

In [118]:
#We sort the dataframe
df = df.sort_index()
lat, lon = df['xlat'], df['xlong']

In [159]:
#We 
df_lat_lon = df[df.index==df.index.unique()[0]][['xlat', 'xlong']].values
locations = np.array(list(zip(df_lat_lon[:,0], df_lat_lon[:,1])))
longitudes = locations[:, 0]  # Extract longitudes
latitudes = locations[:, 1]   # Extract latitudes


In [146]:
time_dim = df.index.unique()
lat = df['xlat'].loc[df.index==time_dim[0]]
lon = df['xlong'].loc[df.index==time_dim[0]]

var1 = 'wsp'
altitude = []
for col in df:
    if var1 in col:
        altitude.append(col.replace(var1+'_', ""))

In [147]:
len_time, time_index = len(df.index.unique()), df.index.unique()
len_lat = len(locations)
len_altitude = len(altitude)

In [148]:
data_array_wsp = np.reshape(df[['wsp_50.0', 'wsp_150.0']], (len_time, len_lat, len_altitude))
data_array_wdir = np.reshape(df[['wdir_50.0', 'wdir_150.0']], (len_time, len_lat, len_altitude))

In [161]:
data_array_wsp = xr.DataArray(data_array_wsp, 
                              dims=["timestamp", "location", "height"],
                              coords={
                                    "time": time_dim,
                                    "location": np.arange(len(locations)),
                                    "longitude": ("location", longitudes),  
                                    "latitude": ("location", latitudes),
                                    "height": altitude
                              })

data_array_wdir = xr.DataArray(data_array_wdir, 
                              dims=["timestamp", "location", "height"],
                              coords={
                                    "time": time_dim,
                                    "location": np.arange(len(locations)),
                                    "longitude": ("location", longitudes),  
                                    "latitude": ("location", latitudes),
                                    "height": altitude
                              })


In [164]:
ds = xr.Dataset({'wind_speed':data_array_wsp,
               'wind_direction': data_array_wdir})

In [165]:
ds

<xarray.Dataset>
Dimensions:         (timestamp: 1464, location: 227, height: 2)
Coordinates:
    time            (timestamp) object '2022-06-01 00:00:00' ... '2022-07-31 ...
  * location        (location) int64 0 1 2 3 4 5 6 ... 221 222 223 224 225 226
    longitude       (location) float64 40.3 40.2 40.1 40.25 ... 40.09 40.44 40.1
    latitude        (location) float64 -74.06 -73.73 -73.66 ... -73.78 -74.03
  * height          (height) <U5 '50.0' '150.0'
Dimensions without coordinates: timestamp
Data variables:
    wind_speed      (timestamp, location, height) float64 0.955 ... 0.9867
    wind_direction  (timestamp, location, height) float64 305.3 313.0 ... 188.3